In [1]:
import os
import sys
sys.path.append('../src')

%load_ext autoreload
%autoreload 2

### Test Different Modules

Input

In [2]:
PATIENT_LIST = [
    775,  787,  788, 1123, 1169, 1170, 1171, 1172, 1173, 1983, 2110, 2195,
    2955, 2956, 2957, 2958, 2959, 2960, 2961, 2962, 2963, 3081, 3229, 3318, 3432
]

Settings

In [3]:
clinical_score_path = "../data/"
protocol_csv_path = "../data/"

In [4]:
max_values = {
    'BARTHEL': 100,
    'ASH_PROXIMAL': 4,
    'MA_DISTAL': 4,
    'FATIGUE': 63,
    'VAS': 10,
    'FM_A': 36,
    'FM_B': 10,
    'FM_C': 14,
    'FM_D': 6,
    'FM_TOTAL': 66,
    'ACT_AU': 10,
    'ACT_QOM': 10
}

latent_to_clinical_mapping_nest = {
    # Functional Independence
    "BARTHEL": ["DAILY_LIVING_ACTIVITY"],  # Barthel Index measures independence in ADLs.

    # Motor Function (Spasticity & Strength)
    "ASH_PROXIMAL": ["BODY_PART_ARM", "BODY_PART_SHOULDER", "COORDINATION"],  # Ashworth scale for proximal limb spasticity.
    "MA_DISTAL": ["BODY_PART_FINGER", "BODY_PART_WRIST", "GRASPING", "PINCHING"],  # Motor Assessment for distal motor function.

    # Fatigue & Pain
    "FATIGUE": ["DIFFICULTY_COGNITIVE", "DIFFICULTY_MOTOR", "PROCESSING_SPEED", "ATTENTION"],  # Fatigue relates to cognitive/motor difficulty.
    "VAS": ["DIFFICULTY_COGNITIVE", "DIFFICULTY_MOTOR"],  # Visual Analog Scale (VAS) for perceived effort.

    # Fugl-Meyer Subscales (Motor Control & Coordination)
    "FM_A": ["BODY_PART_ARM", "BODY_PART_SHOULDER", "RANGE_OF_MOTION_H", "RANGE_OF_MOTION_V"],  # Upper Limb Motor
    "FM_B": ["BODY_PART_WRIST", "PRONATION_SUPINATION", "RANGE_OF_MOTION_H"],  # Wrist Motor
    "FM_C": ["BODY_PART_FINGER", "GRASPING", "PINCHING"],  # Hand Motor
    "FM_D": ["COORDINATION", "RANGE_OF_MOTION_H", "RANGE_OF_MOTION_V"],  # Coordination & Speed
    "FM_TOTAL": ["BODY_PART_ARM", "BODY_PART_WRIST", "BODY_PART_FINGER", "COORDINATION"],  # Full Upper Limb Score

    # Activity & Movement Quality
    "ACT_AU": ["BODY_PART_TRUNK"],  # Activity Autonomy linked to balance.
    "ACT_QOM": ["COORDINATION"],  # Quality of Movement related to balance & coordination.
}

### Pipeline

Initialize Pipelinne

In [46]:
from ai_cdss.services.pipeline import PipelineBase

In [47]:
pipeline = PipelineBase(
    patient_list = PATIENT_LIST,
    clinical_score_path=clinical_score_path, 
    protocol_csv_path= protocol_csv_path, 
    mapping_dict = latent_to_clinical_mapping_nest, 
    max_subscales = max_values
)

Load Data 

In [48]:
pipeline.load_data()

Database engine created successfully
Data successfully saved to rgs_interaction.csv
Database engine closed


In [ ]:
pipeline.

In [ ]:
n = 10
top_n_protocols = (
    pipeline.prescriptions.groupby('PATIENT_ID')
    .apply(lambda x: x.nlargest(n, 'Score'), include_groups=True)
    .reset_index(drop=True)
)
top_n_protocols


In [ ]:
# Find top n protocols by score per patient_id
scores.groupby("PATIENT_ID").apply(lambda x: x.nlargest(1, "Score"))

In [ ]:
pipeline.data_processor.get_protocol(protocol_path="../../data/protocol_attributes.csv")

In [ ]:
from ai_cdss.services.prescription import PrescriptionRecommender
pipeline.prescription_recommender = PrescriptionRecommender()

In [ ]:
pipeline.scores

In [ ]:
rank = pipeline.prescription_recommender.rank_protocols(pipeline.prescriptions)

In [ ]:
schedule.columns

In [ ]:
result = pipeline.prescription_recommender.recommend_protocols(pipeline.prescriptions)

In [ ]:
result

In [ ]:
grouped = result.groupby("PATIENT_ID").apply(lambda g: g.to_dict(orient="records")).reset_index(name="PROTOCOLS")


In [ ]:
import msgspec
import pandas as pd
from typing import List

class ProtocolRecommendation(msgspec.Struct):
    PATIENT_ID: int
    PROTOCOL_ID: int
    PPF: float
    ADHERENCE_EWMA: float
    PARAMETER_VALUE_EWMA: float
    PERFORMANCE_VALUE_EWMA: float
    CONTRIBUTION: float
    Score: float
    DAYS: List[int]

class PrescriptionRecommendation(msgspec.Struct):
    PATIENT_ID: int
    PROTOCOLS: List[ProtocolRecommendation]

    @classmethod
    def from_df(cls, df: pd.DataFrame) -> List["PrescriptionRecommendation"]:
        """
        Convert a long-format DataFrame into a list of PrescriptionRecommendation objects.
        """
        grouped = df.groupby("PATIENT_ID").apply(lambda g: g.to_dict(orient="records")).reset_index(name="PROTOCOLS")

        return [
            cls(
                PATIENT_ID=row.PATIENT_ID,
                PROTOCOLS=[ProtocolRecommendation(**p) for p in row.PROTOCOLS]
            )
            for _, row in grouped.iterrows()
        ]

# Apply conversion
prescription_recommendations = PrescriptionRecommendation.from_df(result)

In [ ]:
prescriptions = pipeline.prescriptions
patient_profile = pipeline.patient_profiles

In [ ]:
patient_profile

In [ ]:
import pandas as pd

# Define the real max values based on provided scales
max_values = {
    'BARTHEL': 100,
    'ASH_PROXIMAL': 4,
    'MA_DISTAL': 4,
    'FATIGUE': 63,
    'VAS': 10,
    'FM_A': 36,
    'FM_B': 10,
    'FM_C': 14,
    'FM_D': 6,
    'FM_TOTAL': 66,
    'ACT_AU': 10,
    'ACT_QOM': 10
}

# Compute the deficit matrix: (1 - normalized score)
deficit_matrix = 1 - (patient_profile / pd.Series(max_values))

In [ ]:
deficit_matrix

In [ ]:
feature_names = list(pipeline.protocol_profiles.columns)

In [ ]:
import numpy as np
# import matplotlib.pyplot as plt

# Initialize total_contr as a zero array with the same shape as the contribution lists
total_contr = np.zeros(12)

# Iterate over each row in the DataFrame and sum the "CONTRIBUTION" column
for i, row in p.iterrows():
    contr = np.asarray(row["CONTRIBUTION"])  # Convert the list to a NumPy array
    total_contr += contr  # Element-wise summation

total_contr  # This is now correctly summed

# Normalize values so they sum to 1
total_contr /= np.sum(total_contr)


In [ ]:
feature_names

In [ ]:
total_contr

In [ ]:
top_n_protocols

In [ ]:
top_n_protocols[["PATIENT_ID", "PROTOCOL_ID"]].pivot(index="PATIENT_ID", columns="PROTOCOL_ID", values="PROTOCOL_ID")

In [ ]:
[{int(patient): top_n_protocols[top_n_protocols.PATIENT_ID == patient].PROTOCOL_ID.values} for patient in top_n_protocols.PATIENT_ID.unique()]

In [ ]:
import pandas as pd
import numpy as np

# Create the dataframe from the provided image description
data = {
    "PATIENT_ID": [775, 775, 775, 775, 775, 787, 787, 787, 787, 787],
    "PROTOCOL_ID": [222, 206, 224, 208, 214, 221, 223, 224, 211, 217],
}

df = pd.DataFrame(data)

# Define parameters
n = 5  # protocols per day
m = 7  # number of days

### Fetch Data

In [ ]:
from recsys_interface.data.interface import fetch_rgs_data, fetch_timeseries_data

In [ ]:
data = fetch_timeseries_data(PATIENT_LIST, "app")

In [ ]:
data

In [ ]:
data.dtypes